# 🏀 Basketball EDA — Qatar Asia Cup Qualifiers
## Lebanon vs India & Saudi Arabia — Scouting Report

This notebook performs exploratory data analysis on player and team stats to identify strengths, weaknesses, and strategic matchup advantages for the Lebanese national basketball team.

---
## 1. Imports & Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ── Styling: clean white background ──────────────────────────────────────────
plt.rcParams.update({
    'figure.facecolor': 'white',
    'axes.facecolor':   '#f8f9fa',
    'axes.edgecolor':   '#cccccc',
    'axes.labelcolor':  '#333333',
    'xtick.color':      '#333333',
    'ytick.color':      '#333333',
    'text.color':       '#333333',
    'grid.color':       '#e0e0e0',
    'grid.linewidth':   0.7,
    'font.family':      'DejaVu Sans',
})

COLORS = {
    'Lebanon': '#C8102E',   # red
    'India':   '#FF9933',   # saffron
    'Saudi':   '#006C35',   # green
}

print('Libraries loaded ✓')

---
## 2. Load & Clean Data

In [ ]:
leb = pd.read_csv('/mnt/user-data/uploads/Leb.csv')
ind = pd.read_csv('/mnt/user-data/uploads/India.csv')
sau = pd.read_csv('/mnt/user-data/uploads/Saudi.csv')

# Strip BOM / whitespace from column names
for df in [leb, ind, sau]:
    df.columns = df.columns.str.strip().str.lstrip('\ufeff')

# Lebanon CSV stores totals → convert to per-game averages
leb_rename = {'MIN': 'MPG', 'PTS': 'PPG', 'REB': 'RPG', 'AST': 'APG',
              'STL': 'SPG', 'BLK': 'BPG'}
leb = leb.rename(columns=leb_rename)
gp_col = leb['GP'].replace(0, 1)
for col in ['PPG','FGM','FGA','3PM','3PA','FTM','FTA','ORB','DRB','RPG','APG','SPG','BPG','TOV','PF','MPG']:
    if col in leb.columns:
        leb[col] = leb[col] / gp_col

# Add team labels
leb['Team'] = 'Lebanon'
ind['Team'] = 'India'
sau['Team'] = 'Saudi'

# Combine & filter out DNP players
all_players = pd.concat([leb, ind, sau], ignore_index=True)
all_players = all_players[all_players['MPG'] > 5].copy()

print(f'Total active players: {len(all_players)}')
all_players[['Player','Team','MPG','PPG','FG%','RPG','APG','SPG','BPG','TOV']].head(10)

---
## 3. Team Aggregate Stats

In [ ]:
def team_agg(df):
    numeric = ['PPG','FGM','FGA','FG%','3PM','3PA','3P%','FTM','FTA','FT%',
               'ORB','DRB','RPG','APG','SPG','BPG','TOV','PF']
    res = {}
    for col in numeric:
        if col in df.columns:
            res[col] = df[col].mean() if col.endswith('%') else df[col].sum()
    return res

teams = {t: team_agg(all_players[all_players['Team'] == t]) for t in ['Lebanon','India','Saudi']}
team_df = pd.DataFrame(teams).T
team_df.index.name = 'Team'

display(team_df[['PPG','FG%','3P%','FT%','RPG','APG','SPG','BPG','TOV']].round(3).style
        .background_gradient(cmap='RdYlGn', axis=0)
        .format({
            'PPG': '{:.1f}', 'FG%': '{:.1%}', '3P%': '{:.1%}',
            'FT%': '{:.1%}', 'RPG': '{:.1f}', 'APG': '{:.1f}',
            'SPG': '{:.1f}', 'BPG': '{:.1f}', 'TOV': '{:.1f}'
        })
        .set_caption('Team Aggregates — Per Game (% columns = average, counting stats = sum)'))

---
## 4. Radar Chart — Team Strengths & Weaknesses

In [ ]:
categories = ['PPG', 'FG%', '3P%', 'FT%', 'RPG', 'APG', 'SPG', 'BPG']
labels     = ['Points', 'FG%', '3P%', 'FT%', 'Rebounds', 'Assists', 'Steals', 'Blocks']
N = len(categories)
angles = np.linspace(0, 2 * np.pi, N, endpoint=False).tolist()
angles += angles[:1]

fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True), facecolor='white')
ax.set_facecolor('#f8f9fa')

for team, color in COLORS.items():
    vals = []
    for cat in categories:
        v = team_df.loc[team, cat]
        mn, mx = team_df[cat].min(), team_df[cat].max()
        vals.append((v - mn) / (mx - mn + 1e-9))
    vals += vals[:1]
    ax.plot(angles, vals, color=color, linewidth=2.5, label=team)
    ax.fill(angles, vals, color=color, alpha=0.12)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(labels, size=12, fontweight='bold', color='#333333')
ax.set_yticklabels([])
ax.set_title('Team Strengths & Weaknesses\n(Normalized per category)',
             size=15, fontweight='bold', color='#222222', pad=25)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.15), fontsize=12)
ax.grid(color='#cccccc', linewidth=0.8)
ax.spines['polar'].set_color('#cccccc')

plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/fig1_radar.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

---
## 5. Shooting Efficiency Comparison

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 6), facecolor='white')
fig.suptitle('Shooting Efficiency Comparison', fontsize=18, fontweight='bold',
             color='#222222', y=1.01)

shoot_cats = [('FG%', 'Field Goal %'), ('3P%', '3-Point %'), ('FT%', 'Free Throw %')]
team_names = ['Lebanon', 'India', 'Saudi']

for ax, (col, title) in zip(axes, shoot_cats):
    vals = [team_df.loc[t, col] * 100 for t in team_names]
    bars = ax.bar(team_names, vals, color=[COLORS[t] for t in team_names],
                  edgecolor='white', linewidth=1, width=0.5)
    for bar, v in zip(bars, vals):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.8,
                f'{v:.1f}%', ha='center', va='bottom', fontsize=12, fontweight='bold', color='#333333')
    ax.set_title(title, fontsize=13, fontweight='bold', color='#222222')
    ax.set_ylim(0, 100)
    ax.set_ylabel('Percentage (%)')
    ax.grid(axis='y', alpha=0.5)
    ax.set_facecolor('#f8f9fa')

plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/fig2_shooting.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

---
## 6. Scoring vs Minutes (Bubble = FG% Efficiency)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8), facecolor='white')
ax.set_facecolor('#f8f9fa')

for team, color in COLORS.items():
    sub = all_players[all_players['Team'] == team]
    sizes = (sub['FG%'].fillna(0) * 500).clip(20, 500)
    ax.scatter(sub['MPG'], sub['PPG'], s=sizes, c=color, alpha=0.75,
               edgecolors='white', linewidths=0.8, label=team, zorder=3)
    for _, row in sub.iterrows():
        ax.annotate(row['Player'].split()[-1], (row['MPG'], row['PPG']),
                    textcoords='offset points', xytext=(6, 4),
                    fontsize=8, color='#333333', alpha=0.9)

ax.set_xlabel('Minutes Per Game', fontsize=12)
ax.set_ylabel('Points Per Game', fontsize=12)
ax.set_title('Scoring vs. Minutes Played\n(Bubble size = FG% efficiency)',
             fontsize=15, fontweight='bold', color='#222222')
ax.legend(fontsize=12)
ax.grid(alpha=0.4, zorder=0)

plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/fig3_scoring_minutes.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

---
## 7. Ball Security — Assists vs Turnovers

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7), facecolor='white')
ax.set_facecolor('#f8f9fa')

for team, color in COLORS.items():
    sub = all_players[all_players['Team'] == team]
    ax.scatter(sub['TOV'], sub['APG'], s=100, c=color, alpha=0.85,
               edgecolors='white', linewidths=0.8, label=team, zorder=3)
    for _, row in sub.iterrows():
        ax.annotate(row['Player'].split()[-1], (row['TOV'], row['APG']),
                    textcoords='offset points', xytext=(5, 4),
                    fontsize=8, color='#333333', alpha=0.9)

med_tov = all_players['TOV'].median()
med_ast = all_players['APG'].median()
ax.axvline(med_tov, color='#999999', linestyle='--', alpha=0.6, lw=1.2)
ax.axhline(med_ast, color='#999999', linestyle='--', alpha=0.6, lw=1.2)

ymax = all_players['APG'].max()
ax.text(med_tov + 0.05, ymax * 0.97, 'High TOV →', color='#888888', fontsize=9)
ax.text(0.02, med_ast + 0.15, 'High AST ↑', color='#888888', fontsize=9)

# Annotate quadrants
ax.text(0.02, 0.02, 'Low AST\nLow TOV\n(Role players)', transform=ax.transAxes,
        fontsize=8, color='#aaaaaa', va='bottom')
ax.text(0.55, 0.97, 'High AST\nHigh TOV\n(Playmakers)', transform=ax.transAxes,
        fontsize=8, color='#aaaaaa', va='top')

ax.set_xlabel('Turnovers Per Game', fontsize=12)
ax.set_ylabel('Assists Per Game', fontsize=12)
ax.set_title('Ball Security: Assists vs Turnovers\n(Best players = top-left quadrant)',
             fontsize=15, fontweight='bold', color='#222222')
ax.legend(fontsize=12)
ax.grid(alpha=0.4, zorder=0)

plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/fig4_tov_ast.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

---
## 8. Defensive & Physical Stats Heatmap

In [ ]:
def_cats   = ['SPG', 'BPG', 'DRB', 'ORB', 'PF', 'TOV']
def_labels = ['Steals/G', 'Blocks/G', 'Def Reb/G', 'Off Reb/G', 'Fouls/G', 'TOV/G']

def_data = pd.DataFrame({
    'Lebanon': [team_df.loc['Lebanon', c] for c in def_cats],
    'India':   [team_df.loc['India', c]   for c in def_cats],
    'Saudi':   [team_df.loc['Saudi', c]   for c in def_cats],
}, index=def_labels)

def_norm = def_data.div(def_data.max(axis=1), axis=0)

fig, ax = plt.subplots(figsize=(10, 6), facecolor='white')
sns.heatmap(def_norm, annot=def_data.round(2), fmt='.2f', cmap='RdYlGn',
            linewidths=0.5, linecolor='white', ax=ax,
            cbar_kws={'label': 'Relative Value (row-normalized)'})
ax.set_title('Defensive & Physical Stats Heatmap\n(Green = Best in category)',
             fontsize=14, fontweight='bold', pad=15, color='#222222')
ax.set_xticklabels(ax.get_xticklabels(), fontsize=12, fontweight='bold')
ax.set_yticklabels(ax.get_yticklabels(), fontsize=11, rotation=0)
ax.figure.set_facecolor('white')

plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/fig5_defense_heatmap.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

---
## 9. Lebanon Player Contributions

In [ ]:
leb_df = all_players[all_players['Team'] == 'Lebanon'].sort_values('PPG', ascending=False)
leb_df = leb_df[leb_df['MPG'] > 5].head(8)

fig, axes = plt.subplots(1, 4, figsize=(18, 6), facecolor='white')
fig.suptitle('Lebanon Player Contributions', fontsize=17, fontweight='bold',
             color='#222222', y=1.01)

metrics = [
    ('PPG', 'Points/G',   '#C8102E'),
    ('RPG', 'Rebounds/G', '#E05070'),
    ('APG', 'Assists/G',  '#FF8C94'),
    ('SPG', 'Steals/G',   '#FFB3B8'),
]

for ax, (col, label, color) in zip(axes, metrics):
    vals = leb_df.set_index('Player')[col].sort_values()
    players_short = [p.split()[0][:10] + ' ' + p.split()[-1][:8] for p in vals.index]
    bars = ax.barh(players_short, vals, color=color, edgecolor='white',
                   linewidth=0.5, alpha=0.9)
    for bar, v in zip(bars, vals):
        ax.text(bar.get_width() + 0.05, bar.get_y() + bar.get_height()/2,
                f'{v:.1f}', va='center', fontsize=9, fontweight='bold', color='#333333')
    ax.set_title(label, fontsize=12, fontweight='bold', color='#222222')
    ax.grid(axis='x', alpha=0.4)
    ax.set_facecolor('#f8f9fa')

plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/fig6_leb_players.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

---
## 10. 3-Point Shot Selection — Volume vs Efficiency

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7), facecolor='white')
ax.set_facecolor('#f8f9fa')

shooters = all_players[all_players['3PA'] >= 1.0]

for team, color in COLORS.items():
    sub = shooters[shooters['Team'] == team]
    ax.scatter(sub['3PA'], sub['3P%'] * 100, s=sub['PPG'] * 14,
               c=color, alpha=0.8, edgecolors='white', linewidths=0.8, label=team, zorder=3)
    for _, row in sub.iterrows():
        ax.annotate(row['Player'].split()[-1], (row['3PA'], row['3P%'] * 100),
                    textcoords='offset points', xytext=(6, 4),
                    fontsize=8, color='#333333', alpha=0.9)

ax.axhline(33.3, color='#888888', linestyle='--', lw=1.5, alpha=0.7, label='33% baseline')
ax.set_xlabel('3-Point Attempts Per Game', fontsize=12)
ax.set_ylabel('3-Point %', fontsize=12)
ax.set_title('3PT Volume vs Efficiency\n(Bubble size = PPG — bigger = higher scorer)',
             fontsize=15, fontweight='bold', color='#222222')
ax.legend(fontsize=11)
ax.grid(alpha=0.4, zorder=0)

plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/fig7_3pt_analysis.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

---
## 11. Scouting Report Dashboard

In [ ]:
fig = plt.figure(figsize=(18, 10), facecolor='white')
fig.suptitle('SCOUTING REPORT — Lebanon vs India & Saudi Arabia\nQatar Asia Cup Qualifiers',
             fontsize=18, fontweight='bold', color='#222222', y=1.00)

# --- (1) Team stat totals grouped bar
ax1 = fig.add_subplot(2, 3, 1)
cats_bar = ['PPG', 'RPG', 'APG', 'SPG', 'BPG']
x = np.arange(len(cats_bar))
w = 0.25
for i, (team, color) in enumerate(COLORS.items()):
    vals = [team_df.loc[team, c] for c in cats_bar]
    ax1.bar(x + i*w, vals, w, label=team, color=color, edgecolor='white', lw=0.5)
ax1.set_xticks(x + w)
ax1.set_xticklabels(cats_bar, fontsize=10)
ax1.set_title('Team Stat Totals', fontsize=12, fontweight='bold')
ax1.legend(fontsize=8)
ax1.grid(axis='y', alpha=0.4)
ax1.set_facecolor('#f8f9fa')

# --- (2) Shooting efficiency
ax2 = fig.add_subplot(2, 3, 2)
eff_cats = ['FG%', '3P%', 'FT%']
for i, (team, color) in enumerate(COLORS.items()):
    vals = [team_df.loc[team, c] * 100 for c in eff_cats]
    ax2.bar(np.arange(len(eff_cats)) + i*0.25, vals, 0.25,
            label=team, color=color, edgecolor='white', lw=0.5)
ax2.set_xticks(np.arange(len(eff_cats)) + 0.25)
ax2.set_xticklabels(eff_cats, fontsize=10)
ax2.set_title('Shooting Efficiency (%)', fontsize=12, fontweight='bold')
ax2.set_ylim(0, 100)
ax2.legend(fontsize=8)
ax2.grid(axis='y', alpha=0.4)
ax2.set_facecolor('#f8f9fa')

# --- (3) Turnovers
ax3 = fig.add_subplot(2, 3, 3)
tov_vals = [team_df.loc[t, 'TOV'] for t in COLORS]
bars = ax3.bar(list(COLORS.keys()), tov_vals,
               color=list(COLORS.values()), edgecolor='white', lw=0.5, width=0.5)
for bar, v in zip(bars, tov_vals):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
             f'{v:.1f}', ha='center', fontsize=11, fontweight='bold', color='#333333')
ax3.set_title('Turnovers Per Game (Lower = Better)', fontsize=12, fontweight='bold')
ax3.grid(axis='y', alpha=0.4)
ax3.set_facecolor('#f8f9fa')

# --- (4) Top 10 scorers
ax4 = fig.add_subplot(2, 3, (4, 5))
top_scorers = all_players.nlargest(10, 'PPG')[['Player', 'Team', 'PPG', 'FG%']]
bar_colors = [COLORS[t] for t in top_scorers['Team']]
names = [p.split()[0][0] + '. ' + p.split()[-1] for p in top_scorers['Player']]
bars = ax4.barh(names, top_scorers['PPG'], color=bar_colors,
                edgecolor='white', linewidth=0.5, alpha=0.9)
for bar, v, fg in zip(bars, top_scorers['PPG'], top_scorers['FG%']):
    ax4.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height()/2,
             f'{v:.1f} pts  ({fg*100:.0f}% FG)', va='center', fontsize=9, color='#333333')
ax4.set_title('Top 10 Scorers Across All Teams', fontsize=12, fontweight='bold')
ax4.set_xlabel('Points Per Game')
ax4.grid(axis='x', alpha=0.4)
ax4.set_facecolor('#f8f9fa')
patches = [mpatches.Patch(color=c, label=t) for t, c in COLORS.items()]
ax4.legend(handles=patches, fontsize=9)

# --- (5) Scouting notes text box
ax5 = fig.add_subplot(2, 3, 6)
ax5.set_facecolor('#fafafa')
ax5.axis('off')

notes = [
    ('LEBANON STRENGTHS', '#C8102E', [
        '• Arakji: elite playmaker + scorer (20 PPG)',
        '• Lawson: dominant rebounder',
        '• Best Free Throw % of three teams',
        '• El Darwich: balanced 2-way guard',
    ]),
    ('INDIA WEAKNESS', '#FF6600', [
        '• Lowest FG% — poor shot selection',
        '• High TOV exposure vs press defense',
        '• 3P% inconsistency team-wide',
    ]),
    ('SAUDI THREAT', '#006C35', [
        '• Abdur-Rahkman: hot scorer (23.5 PPG)',
        "• Elite FT% (93.3%) — foul trouble costly",
        '• Strong interior defense (4 BPG)',
    ]),
]

y = 0.98
for title, color, bullets in notes:
    ax5.text(0.02, y, title, transform=ax5.transAxes,
             fontsize=10, fontweight='bold', color=color, va='top')
    y -= 0.08
    for b in bullets:
        ax5.text(0.04, y, b, transform=ax5.transAxes,
                 fontsize=8.5, color='#333333', va='top')
        y -= 0.07
    y -= 0.03

plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/fig8_dashboard.png', dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

---
## 12. Key Scouting Insights Summary

In [ ]:
print('=' * 60)
print('KEY SCOUTING INSIGHTS')
print('=' * 60)

print('\n🇱🇧 LEBANON STRENGTHS:')
print(f"  - Highest scoring team: {team_df.loc['Lebanon','PPG']:.1f} PPG")
print(f"  - Best FT%: {team_df.loc['Lebanon','FT%']*100:.1f}%")
print(f"  - Most assists: {team_df.loc['Lebanon','APG']:.1f} APG → excellent ball movement")
arakji = leb[leb['Player'].str.contains('Arakji')]
if not arakji.empty:
    print(f"  - Arakji: {arakji['PPG'].values[0]:.1f} PPG + {arakji['APG'].values[0]:.1f} APG")

print('\n🇱🇧 LEBANON WEAKNESSES:')
print(f"  - 3P%: {team_df.loc['Lebanon','3P%']*100:.1f}% — needs improvement")
print(f"  - TOV: {team_df.loc['Lebanon','TOV']:.1f} per game — ball security risk")

print('\n🇮🇳 vs INDIA — HOW TO WIN:')
print(f"  - India FG%: {team_df.loc['India','FG%']*100:.1f}% (lowest of 3 teams) — let them shoot")
print(f"  - India 3P%: {team_df.loc['India','3P%']*100:.1f}% — give them open 3s")
print(f"  - Force turnovers: India TOV = {team_df.loc['India','TOV']:.1f} — apply press")
print(f"  - Lebanon rebounding edge: {team_df.loc['Lebanon','RPG']:.1f} vs {team_df.loc['India','RPG']:.1f} RPG")

print('\n🇸🇦 vs SAUDI — STRATEGY:')
print(f"  - Abdur-Rahkman is the key threat — bait him into foul trouble early")
print(f"  - Saudi FT% = 93.3% — avoid fouling him at all costs")
print(f"  - Saudi TOV = {team_df.loc['Saudi','TOV']:.1f} — pressing defense can pay off")
print(f"  - Watch Saudi blocks ({team_df.loc['Saudi','BPG']:.1f} BPG) — avoid straight-line drives")
print(f"  - Lebanon must exploit superior FT shooting in clutch moments")